In [7]:
import pandas as pd
import numpy as np
import scipy as sp

In [90]:
# import sys
# !{sys.executable} -m pip install BayesPy

In [91]:
import seaborn as sns
from matplotlib import pyplot as plt

In [96]:
lamda=np.random.gamma(np.random.random(),np.random.random(),2)
lamda

array([0.58280303, 0.16172482])

In [108]:
squarer = lambda t: sp.special.digamma(t) 
vfunc = np.vectorize(squarer)
a=vfunc(np.random.rand(1,3))
a[0]

array([ -7.01169135, -11.48377162, -13.71896176])

In [253]:
b=sp.special.digamma(7/2)
b

1.103156640645243

In [124]:
x=np.array([[2,3,4,5],[1,2,3,4]])
x[:,1].sum()

5

In [446]:
import numpy as np

class UGMM(object):
    '''Univariate GMM with CAVI'''
    def __init__(self, X, K=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]
#         self.sigma2 = sigma**2

    def _init(self):
#         self.pi = np.random.dirichlet([np.random.random()*np.random.randint(1, 10)]*self.K, self.N)
#         self.m = np.random.random((self.X.min()), (self.X.max()), size=self.K).astype(float)
        self.alpha=np.random.rand(self.K,1)
        self.m= np.random.rand(self.K,1)
        
        self.beta=np.random.rand(self.K,1)
        self.beta_prev=np.random.rand(self.K,1)
        self.nu=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.W=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.alpha=np.random.rand(self.K,1)
        self.r=np.zeros((self.N,self.K))
        self.x_nebla=np.zeros((self.K,1))
        self.Nk=np.zeros((self.K,1))
        self.S=np.zeros((self.K,1))
        self.pi_nebla=np.zeros((self.K,1))
        self.lamda_nebla=np.zeros((self.K,1))
        self.E_mu_lamda=np.zeros((self.N,self.K))
        
        
#         self.s2 = np.ones(self.K) * np.random.random(self.K)
        print('Init mean')
        print(self.m)
        print('Init lamda')
#         print(self.lamda)

    def get_elbo(self):
        for n in range(self.N):
            for k in range(self.K):
                self.pi_nebla[k]=np.exp(sp.special.digamma(self.alpha[k])-sp.special.digamma(self.alpha.sum()))
                self.lamda_nebla[k]=np.exp(sp.special.digamma(self.nu[k]/2, out=None)+np.log(2)+np.log(self.W[k]))
#                 print(self.m)
                self.E_mu_lamda[n][k]=(self.beta[k]**(-1))+self.nu[k]*self.W[k]*(self.X[n]-self.m[k])**2
                self.r[n][k]=self.pi_nebla[k]*(self.lamda_nebla[k]**0.5)*(np.exp(-0.5*self.E_mu_lamda[n][k]))
        row_sums = self.r.sum(axis=1)
        self.r= self.r / row_sums[:, np.newaxis]
#         return r
    def fit(self, max_iter=1000, tol=1e-6):
        self._init()
#         self.r_values = [self.r_elbo()]
        self.m_history = [self.m.copy()]
        self.get_elbo()
#         self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
#             print('before')
#             print(self.m_history)
            self._cavi()
#             print('after')
#             print(self.m_history)
            self.m_history.append(self.m.copy())
#             if iter_ % 2 == 0:
#             print('wow')
            print(iter_, self.m_history[iter_])
            print(iter_, self.W)
            if np.linalg.norm(self.m_history[-1]-self.m_history[-2],2) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.m_history[-1][0],iter_))
                break
            self.get_elbo()
        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.m_history[-1][0]))


    def _cavi(self):
        self._update_Nk()
        self._update_x_nebla()
        self._update_S()
        self._update_alpha()
        self._update_W()
        self._update_beta()
        self._update_nu()
        self._update_m()

    def _update_Nk(self):
        for k in range(self.K):
            self.Nk[k]=self.r[:,k].sum()
    def _update_x_nebla(self):
        for k in range(self.K):
            self.x_nebla[k]=0
            for n in range(self.N):
#                 print(self.Nk)
                self.x_nebla[k]+=self.r[n][k]*self.X[n]/self.Nk[k]
    def _update_S(self):
        for k in range(self.K):
            self.S[k]=0
            for n in range(self.N):
                self.S[k]+=(self.r[n][k]*(self.X[n]-self.x_nebla[k])**(2))/self.Nk[k]
    def _update_alpha(self):
        for k in range(self.K):
            self.alpha[k]=self.alpha[k]+self.Nk[k]
    def _update_beta(self):
        for k in range(self.K):
            self.beta[k]=self.beta[k]+self.Nk[k]
        self.beta_prev=self.beta.copy()
    def _update_m(self):
        for k in range(self.K):
            self.m[k]=(self.beta_prev[k]*self.m[k]+self.Nk[k]*self.x_nebla[k])/self.beta[k]
#         print('achha')
#         print(self.m)
    
    
    def _update_W(self):
        for k in range(self.K):
            self.W[k]=((1/self.W[k])+self.Nk[k]*self.S[k]+(self.beta[k]*self.Nk[k]/(self.beta[k]+self.Nk[k]))*(self.x_nebla[k]-self.m[k])**2)**(-1)
        
    
    def _update_nu(self):
        for k in range(self.K):
            self.nu[k]=self.nu[k]+self.Nk[k]
    
#     def _calculate_rnk
        

In [447]:
data=pd.read_csv('data2.txt',header=None)

In [448]:
X =np.array(data.iloc[0,0])
for i in range(1,1000):
    X= np.append(X, data.iloc[i,0])
    

In [449]:
X.max()

0.9942949885119834

In [ ]:
ugmm = UGMM(X, 2)
ugmm.fit()

Init mean
[[0.38023865]
 [0.94569573]]
Init lamda
1 [[0.77662132]
 [1.43823325]]
1 [[0.09687873]
 [0.06696655]]
2 [[1.09440645]
 [1.43841191]]
2 [[0.01463313]
 [0.06685006]]
3 [[1.28105116]
 [1.43847003]]
3 [[0.00296483]
 [0.0667893 ]]
4 [[1.41316431]
 [1.43853   ]]
4 [[0.00117603]
 [0.06673564]]
5 [[1.5154063]
 [1.4385868]]
5 [[0.00062825]
 [0.06668749]]
6 [[1.59879423]
 [1.43863965]]
6 [[0.00039364]
 [0.06664382]]
7 [[1.66919914]
 [1.43868878]]
7 [[0.00027195]
 [0.06660384]]
8 [[1.73011918]
 [1.43873461]]
8 [[0.00020058]
 [0.06656693]]
9 [[1.78380653]
 [1.43877755]]
9 [[0.00015501]
 [0.0665326 ]]
10 [[1.83179634]
 [1.43881796]]
10 [[0.00012402]
 [0.06650046]]
11 [[1.87518188]
 [1.43885615]]
11 [[0.00010193]
 [0.06647023]]
12 [[1.91476931]
 [1.43889237]]
12 [[8.55809743e-05]
 [6.64416549e-02]]
13 [[1.95117009]
 [1.43892684]]
13 [[7.31074954e-05]
 [6.64145455e-02]]
14 [[1.98485903]
 [1.43895975]]
14 [[6.33523476e-05]
 [6.63887371e-02]]
15 [[2.01621219]
 [1.43899123]]
15 [[5.55628459e-0

116 [[2.94315972]
 [1.44032083]]
116 [[2.02290903e-06]
 [6.53475872e-02]]
117 [[2.94704387]
 [1.44032812]]
117 [[1.99793846e-06]
 [6.53420933e-02]]
118 [[2.95089494]
 [1.44033536]]
118 [[1.97350029e-06]
 [6.53366356e-02]]
119 [[2.95471349]
 [1.44034255]]
119 [[1.94957856e-06]
 [6.53312134e-02]]
120 [[2.95850006]
 [1.4403497 ]]
120 [[1.92615791e-06]
 [6.53258263e-02]]
121 [[2.9622552]
 [1.4403568]]
121 [[1.90322359e-06]
 [6.53204738e-02]]
122 [[2.9659794 ]
 [1.44036386]]
122 [[1.88076138e-06]
 [6.53151554e-02]]
123 [[2.96967318]
 [1.44037088]]
123 [[1.85875761e-06]
 [6.53098706e-02]]
124 [[2.97333704]
 [1.44037785]]
124 [[1.83719912e-06]
 [6.53046189e-02]]
125 [[2.97697145]
 [1.44038478]]
125 [[1.81607322e-06]
 [6.52993999e-02]]
126 [[2.98057688]
 [1.44039167]]
126 [[1.79536770e-06]
 [6.52942131e-02]]
127 [[2.9841538 ]
 [1.44039851]]
127 [[1.77507076e-06]
 [6.52890581e-02]]
128 [[2.98770264]
 [1.44040532]]
128 [[1.75517108e-06]
 [6.52839344e-02]]
129 [[2.99122385]
 [1.44041208]]
129 [[1

228 [[3.24888547]
 [1.44094166]]
228 [[7.77808607e-07]
 [6.48816043e-02]]
229 [[3.25086515]
 [1.440946  ]]
229 [[7.73109838e-07]
 [6.48783619e-02]]
230 [[3.25283619]
 [1.44095032]]
230 [[7.68460898e-07]
 [6.48751308e-02]]
231 [[3.25479869]
 [1.44095463]]
231 [[7.63861037e-07]
 [6.48719108e-02]]
232 [[3.2567527 ]
 [1.44095893]]
232 [[7.59309518e-07]
 [6.48687018e-02]]
233 [[3.25869831]
 [1.4409632 ]]
233 [[7.54805620e-07]
 [6.48655038e-02]]
234 [[3.26063558]
 [1.44096747]]
234 [[7.50348636e-07]
 [6.48623167e-02]]
235 [[3.26256459]
 [1.44097172]]
235 [[7.45937872e-07]
 [6.48591403e-02]]
236 [[3.2644854 ]
 [1.44097596]]
236 [[7.41572646e-07]
 [6.48559747e-02]]
237 [[3.26639809]
 [1.44098018]]
237 [[7.37252290e-07]
 [6.48528197e-02]]
238 [[3.26830272]
 [1.44098439]]
238 [[7.32976151e-07]
 [6.48496753e-02]]
239 [[3.27019937]
 [1.44098858]]
239 [[7.28743583e-07]
 [6.48465413e-02]]
240 [[3.27208809]
 [1.44099276]]
240 [[7.24553958e-07]
 [6.48434177e-02]]
241 [[3.27396896]
 [1.44099693]]
241 [

341 [[3.43096449]
 [1.44135827]]
341 [[4.48150433e-07]
 [6.45709742e-02]]
342 [[3.43228896]
 [1.44136144]]
342 [[4.46375292e-07]
 [6.45686218e-02]]
343 [[3.43360955]
 [1.44136459]]
343 [[4.44612588e-07]
 [6.45662749e-02]]
344 [[3.43492631]
 [1.44136774]]
344 [[4.42862197e-07]
 [6.45639334e-02]]
345 [[3.43623924]
 [1.44137088]]
345 [[4.41123995e-07]
 [6.45615975e-02]]
346 [[3.43754837]
 [1.44137402]]
346 [[4.39397862e-07]
 [6.45592670e-02]]
347 [[3.43885372]
 [1.44137714]]
347 [[4.37683678e-07]
 [6.45569418e-02]]
348 [[3.44015531]
 [1.44138026]]
348 [[4.35981325e-07]
 [6.45546221e-02]]
349 [[3.44145317]
 [1.44138337]]
349 [[4.34290687e-07]
 [6.45523077e-02]]
350 [[3.44274731]
 [1.44138648]]
350 [[4.32611649e-07]
 [6.45499987e-02]]
351 [[3.44403776]
 [1.44138958]]
351 [[4.30944097e-07]
 [6.45476949e-02]]
352 [[3.44532454]
 [1.44139267]]
352 [[4.29287919e-07]
 [6.45453964e-02]]
353 [[3.44660767]
 [1.44139575]]
353 [[4.27643005e-07]
 [6.45431032e-02]]
354 [[3.44788716]
 [1.44139883]]
354 [

452 [[3.55842027]
 [1.44167159]]
452 [[3.06651100e-07]
 [6.43383454e-02]]
453 [[3.55941979]
 [1.44167412]]
453 [[3.05745965e-07]
 [6.43364709e-02]]
454 [[3.5604171 ]
 [1.44167665]]
454 [[3.04845578e-07]
 [6.43345997e-02]]
455 [[3.56141221]
 [1.44167917]]
455 [[3.03949903e-07]
 [6.43327318e-02]]
456 [[3.56240514]
 [1.44168168]]
456 [[3.03058904e-07]
 [6.43308673e-02]]
457 [[3.5633959 ]
 [1.44168419]]
457 [[3.02172546e-07]
 [6.43290060e-02]]
458 [[3.56438449]
 [1.4416867 ]]
458 [[3.01290796e-07]
 [6.43271481e-02]]
459 [[3.56537092]
 [1.4416892 ]]
459 [[3.00413619e-07]
 [6.43252935e-02]]
460 [[3.56635521]
 [1.4416917 ]]
460 [[2.99540980e-07]
 [6.43234421e-02]]
461 [[3.56733735]
 [1.4416942 ]]
461 [[2.98672847e-07]
 [6.43215940e-02]]
462 [[3.56831737]
 [1.44169669]]
462 [[2.97809186e-07]
 [6.43197491e-02]]
463 [[3.56929527]
 [1.44169917]]
463 [[2.96949963e-07]
 [6.43179075e-02]]
464 [[3.57027106]
 [1.44170165]]
464 [[2.96095148e-07]
 [6.43160690e-02]]
465 [[3.57124475]
 [1.44170413]]
465 [

In [425]:
ugmm.beta

array([[nan],
       [nan],
       [nan]])

In [89]:
sorted(ugmm.m)

[0.4508884331910655, 0.4508884337089777, 0.45088843539735585]

In [396]:
a=np.array([1,2,3])

In [397]:
a[1]

2